# Compound Interest Calculator

This notebook calculates and visualizes compound interest growth including periodic contributions.

Heavily inspired by https://finary.com/en/tools/compound-interests-calculator.
*This tool is for information purposes only. It should not be considered as financial advice.*

In [ ]:
import numpy as np
from ipecharts import EChartsRawWidget
from ipywidgets import widgets
from IPython.display import display

In [ ]:
def calculate_compound_interest(initial_amount, monthly_savings, years, annual_rate, compounding_months):
    """
    Calculate compound interest with regular contributions
    """
    # Convert annual rate to decimal
    rate = annual_rate / 100
    
    # Calculate rate per period
    rate_per_period = rate * (compounding_months / 12)
    
    # Total number of periods
    total_periods = int(years * (12 / compounding_months))
    
    # Monthly savings converted to per-period savings
    period_savings = monthly_savings * compounding_months
    
    # Initialize arrays
    balances = []
    contributions = []
    interests = []
    
    current_balance = initial_amount 
    total_contributions = initial_amount
    
    # Calculate growth for each period
    for period in range(total_periods + 1):
        balances.append(current_balance)
        contributions.append(total_contributions)
        interests.append(current_balance - total_contributions)
        
        # Calculate interests
        curr_interests = rate_per_period * current_balance 
        # Add periodic savings
        current_balance += period_savings + curr_interests
        # Keep track of total contributions
        total_contributions += period_savings
    
    # Create time points (in years)
    time_points = np.linspace(0, years, len(balances))
    
    return time_points, balances, contributions, interests

In [ ]:
def create_investment_plot_option(initial_amount, monthly_savings, years, annual_rate, compounding_frequency):
    """
    Create an interactive plot showing investment growth over time
    """
    time_points, balances, contributions, interests = calculate_compound_interest(
        initial_amount, monthly_savings, years, annual_rate, compounding_frequency
    )

    xls = time_points.tolist()

    contributions_name = 'Contributions'
    interests_name = 'Interests'
    contributions_color = 'rgb(100, 140, 200)'
    interests_color = 'rgb(180, 151, 111)'

    option = {
        'title': [
            {
                'text': 'Final Capital',
                'subtext': f'{f"{balances[-1]:,.0f}".replace(",", " ")} €',
                'left': '50px',
                'top': '5%',
                'textStyle': {
                    'color': '#fff',
                    'fontSize': 14,
                },
                'subtextStyle': {
                    'color': '#fff',
                    'fontSize': 24,
                    'fontWeight': 'bold'
                }
            },
            {
                'text': contributions_name,
                'subtext': f"{contributions[-1]:,.0f} €".replace(",", " "),
                'left': '200px',
                'top': '5%',
                'textStyle': {
                    'color': contributions_color,
                    'fontSize': 14,
                },
                'subtextStyle': {
                    'color': contributions_color,
                    'fontSize': 24,
                    'fontWeight': 'bold'
                }
            },
            {
                'text': interests_name,
                'subtext': f"{interests[-1]:,.0f} €".replace(",", " "),
                'left': '350px',
                'top': '5%',
                'textStyle': {
                    'color': interests_color,
                    'fontSize': 14,
                },
                'subtextStyle': {
                    'color': interests_color,
                    'fontSize': 24,
                    'fontWeight': 'bold'
                }
            },
        ],
        'legend': {
            'data': [contributions_name, interests_name],
            'orient': 'vertical',
            'right': '50px',
            'top': '50px',
            'textStyle': {
                'color': '#fff',
                'fontSize': 14
            }
        },
        'tooltip': {
            'trigger': 'axis',
            'axisPointer': {
                'type': 'line',
                'label': {
                    'backgroundColor': '#1a1a1a'
                }
            },
            'formatter': f'''
                 <div style="display: flex; justify-content: space-between; font-size: 14px; color: white; font-weight: 500; margin: 8px 0;"">
                    <span>Year {{b}}</span>
                    <span>{{c0}} €</span>
                </div>
                <div style="display: flex; justify-content: space-between; color: rgb(180, 151, 111); margin: 4px 0;">
                    <span>{contributions_name}</span>
                    <span>{{c2}} €</span>
                </div>
                <div style="display: flex; justify-content: space-between; gap: 10px; color: rgb(100, 140, 200); margin: 4px 0;">
                    <span>Contributions</span>
                    <span>{{c1}} €</span>
                </div>
            ''',
            'backgroundColor': 'rgba(26, 26, 26, 0.9)',
            'borderColor': '#333',
            'padding': [10, 15],
            'textStyle': {
                'color': '#fff',
                'fontSize': 14
            },
        },
        'grid': {
            'left': '3%',
            'right': '5%',
            'bottom': '5%',
            'top': '25%',
            'containLabel': True
        },
        'xAxis': {
            'type': 'category',
            'name': 'Years',
            'boundaryGap' : False,
            'axisLabel': {
                'color': '#fff',
                'fontSize': 12
            },
            'nameTextStyle': {
              'align': 'center',
              'padding': [-20, 0, 0, 20],
            },
            'axisLine': {
                'lineStyle': {
                    'color': '#fff',
                    'width': 1
                }
            },
            'data': xls
        },
        'yAxis': {
            'type': 'value',
            'axisLabel': {
                'color': '#fff',
                'fontSize': 12,
                'formatter': '{value} €'
            },
            'splitLine': {
                'show': True,
                'lineStyle': {
                    'color': 'rgba(255, 255, 255, 0.1)',
                    'type': 'dashed'
                }
            },
            'axisLine': {
                'lineStyle': {
                    'color': '#fff',
                    'width': 1
                }
            }
        },
        'series': [
            {
                'name': 'Total',
                'type': 'line',
                'data': [round(c + i) for c, i in zip(contributions, interests)],
                'symbol': 'none',
                'lineStyle': {'width': 0},
                'areaStyle': {'opacity': 0},
            },
            {
                'name': contributions_name,
                'type': 'line',
                'stack': 'Total',
                'areaStyle': {
                    'opacity': 0.7,
                    'color': contributions_color
                },
                'symbol': 'circle',
                'symbolSize': 8,
                'showSymbol': False,
                'emphasis': {
                    'focus': 'series',
                    'scale': 1.5,
                    'blurScope': 'coordinateSystem'
                },
                'data': [round(c) for c in contributions],
                'itemStyle': {
                    'color': contributions_color
                },
                'smooth': 0.3
            },
            {
                'name': interests_name,
                'type': 'line',
                'stack': 'Total',
                'areaStyle': {
                    'opacity': 0.7,
                    'color': interests_color
                },
                'symbol': 'circle',
                'symbolSize': 8,
                'showSymbol': False,
                'emphasis': {
                    'focus': 'series',
                    'scale': 1.5,
                    'blurScope': 'coordinateSystem'
                },
                'data': [round(i) for i in interests],
                'itemStyle': {
                    'color': interests_color
                },
                'smooth': 0.3
            }
        ],
        'backgroundColor': '#1a1a1a',
        'textStyle': {
            'color': '#fff'
        }
    }
    
    return option

In [ ]:
custom_style = """
<style>
.calculator-container {
    background: var(--jp-layout-color0);
    padding: 30px;
    border-radius: var(--jp-border-radius);
    color: var(--jp-content-font-color1);
    font-family: var(--jp-ui-font-family);
}

.parameter-label {
    display: flex;
    justify-content: space-between;
    align-items: center;
    opacity: 0.7;
    font-size: var(--jp-ui-font-size1);
    margin-bottom: 4px;
}

.parameter-unit {
    text-align: right;
    font-size: var(--jp-ui-font-size0);
    opacity: 0.5;
    margin-top: 10px;
}

.info-icon {
    opacity: 0.5;
    cursor: help;
    color: var(--jp-content-font-color2);
}

/* Hide default number input spinners */
input[type=number]::-webkit-inner-spin-button, 
input[type=number]::-webkit-outer-spin-button { 
    -webkit-appearance: none; 
    margin: 0; 
}
input[type=number] {
    -moz-appearance: textfield;
}

.widget-slider {
    display: none !important;
}

.widget-text {
    border: none !important;
    background: transparent !important;
    box-shadow: none !important;
    margin: 0 !important;
    padding: 0 !important;
    height: 40px !important;
}

.widget-text input {
    font-size: var(--jp-ui-font-size3) !important;
    font-weight: 400 !important;
    background: transparent !important;
    border: none !important;
    padding: 0 !important;
    margin: 0 !important;
    height: 40px !important;
    line-height: 40px !important;
    width: 100% !important;
    color: var(--jp-content-font-color0) !important;
}

.jp-OutputArea-output {
    padding: 0 !important;
}

.parameter-group {
    border-bottom: 1px solid var(--jp-border-color2);
}

/* Hide description labels from ipywidgets */
.widget-label {
    display: none !important;
}

/* Dark mode specific adjustments */
[data-jp-theme-light="false"] .calculator-container {
    background: var(--jp-layout-color1);
}

[data-jp-theme-light="false"] .widget-text input {
    color: var(--jp-content-font-color1) !important;
}

#rendered_cells > div {
    max-width: 1400px;
    margin: auto;
}

</style>
"""

# Create layout configurations
layout = widgets.Layout(width='100%', margin='0', padding='10px')

# Create widgets with improved styling
def create_parameter_group(label, unit, info_tooltip=""):
    info_icon = "ⓘ" if info_tooltip else ""
    return f"""
    <div class="parameter-group">
        <div class="parameter-label">
            {label} {f'<span class="info-icon" title="{info_tooltip}">{info_icon}</span>' if info_tooltip else ''}
        </div>
    """

def create_unit(unit):
    return f'<div class="parameter-unit">{unit}</div>'

initial_amount = widgets.IntText(
    value=10000,
    layout=layout,
)

monthly_savings = widgets.IntText(
    value=100,
    layout=layout,
)

years = widgets.IntText(
    value=20,
    layout=layout,
)

annual_rate = widgets.FloatText(
    value=5,
    layout=layout,
)

compounding_frequency = widgets.IntText(
    value=12,
    layout=layout,
)

CURRENCY = "EUR"

# Create input container with new styling
input_container = widgets.VBox([
    widgets.HTML(create_parameter_group("Initial capital", CURRENCY, "Starting investment amount")),
    widgets.HBox([initial_amount, widgets.HTML(create_unit(CURRENCY))]),
    widgets.HTML(create_parameter_group("Monthly contribution", CURRENCY, "Amount to invest each month")),
    widgets.HBox([monthly_savings, widgets.HTML(create_unit(CURRENCY))]),
    widgets.HTML(create_parameter_group("Years of growth", "YEARS")),
    widgets.HBox([years, widgets.HTML(create_unit("YEARS"))]),
    widgets.HTML(create_parameter_group("Interest rate", "%", "Annual interest rate")),
    widgets.HBox([annual_rate, widgets.HTML(create_unit("%"))]),
    widgets.HTML(create_parameter_group("Compound frequency", "MONTHS", "How often interest is compounded")),
    widgets.HBox([compounding_frequency, widgets.HTML(create_unit("MONTHS"))])
], layout=widgets.Layout(
    min_width='300px',
    max_width='400px',
    width='auto',
    height='500px',
    padding='15px'
))

input_container.add_class('calculator-container')

chart = EChartsRawWidget(
    # width='800px',
    height='500px'
)

# Create main container with horizontal layout
main_container = widgets.HBox([
    # Left panel - Parameters
    input_container,
    
    # Right panel - Chart
    widgets.VBox([
        chart
    ], layout=widgets.Layout(
        min_width='300px',
        flex='1 1 500px',
        align_self='stretch',
        margin='0'
    ))
], layout=widgets.Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='flex-start',
    width='100%'
))


# Create update function
def update_plot(change=None):
    option = create_investment_plot_option(
        initial_amount.value,
        monthly_savings.value,
        years.value,
        annual_rate.value,
        compounding_frequency.value
    )
    chart.option = option

# Link widgets to update function
for w in [initial_amount, monthly_savings, years, annual_rate, compounding_frequency]:
    w.observe(update_plot, names='value')

# Display everything
display(widgets.HTML(custom_style))
display(main_container)
update_plot()